So the reason i am trying to do it with elastic search is that docker doesnt work with my pc, it just crashes. 

I also ran into problems when importing txtai, even after installing everything from scratch, so I decided to try in colab, and there it works. I am now trying to upload from colab to the local elastic search. 

Got same problem when trying to access the pubmedBERT from transformers, so dont know what i did wrong, pls try and see if it works for you :)

In [ ]:
# !pip install --upgrade pip
# !pip install txtai
# !pip install --upgrade numpy transformers # to not get an error when importing txtai, but didnt work
!pip install elasticsearch transformers torch pandas



In [20]:
import pandas as pd
# import txtai # dont know why is doesnt work in this env when i have installed everything in the notebook
# import transformers
# import torch

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

from sentence_transformers import SentenceTransformer # dont know why any of them work, think my env is f-ed



ImportError: cannot import name 'get_token' from 'huggingface_hub.utils' (c:\Users\isakb\Anaconda3\lib\site-packages\huggingface_hub\utils\__init__.py)

In [22]:
df = pd.read_csv("pubmed_articles_first_9999.csv")

print(df.head())
print(len(df))

       PMID                                              Title  \
0  38336420  The impact of austerity on children: Uncoverin...   
1  38333244  Artificial intelligence-powered intraoperative...   
2  38333196  Helix-based screening with structure predictio...   
3  38333180  Role of robotics and artificial intelligence i...   
4  38333148  Relationship between spiritual intelligence an...   

                                            Abstract Publication Year  
0  Which children are most vulnerable when their ...             2024  
1                              No abstract available             2024  
2  The rapid development of drugs against emergin...             2024  
3  Artificial intelligence or AI may be identifie...             2023  
4  The COVID-19 pandemic has caused physical and ...             2023  
9997


## Preproccessing

Expand later

In [23]:
df.isna().value_counts()


PMID   Title  Abstract  Publication Year
False  False  False     False               9835
              True      False                140
       True   False     False                 20
              True      False                  2
Name: count, dtype: int64

In [24]:
df.fillna("None", inplace=True)

__This is the performance of the different models on huggingface, so use the pubmed bert__

Model	PubMed QA	PubMed Subset	PubMed Summary	Average

all-MiniLM-L6-v2	90.40	95.86	94.07	93.44

bge-base-en-v1.5	91.02	95.60	94.49	93.70

gte-base	92.97	96.83	96.24	95.35

pubmedbert-base-embeddings	93.27	97.07	96.58	95.64

S-PubMedBert-MS-MARCO	90.86	93.33	93.54	92.58

In [ ]:
# Function to prepare documents from DataFrame
def documents(df):
    # Combine title and abstract for the text content, use PMID as the document ID
    return [{"id": str(row["PMID"]), "text": f"{row['Title']} {row['Abstract']}"} for index, row in df.iterrows()]

# Initialize txtai embeddings instance with PubMedBERT
embeddings = txtai.Embeddings({"path": "neuml/pubmedbert-base-embeddings", "content": True})


In [ ]:

# Index the documents
embeddings.index(documents(df))

# Example search query
results = embeddings.search("Artificial intelligence in healthcare", limit=5)

# Display search results
for result in results:
    # result = (id, score)
    # Fetch the document using result[0] as the index into df
    document = df.loc[df['PMID'] == int(result[0])]
    print(f"PMID: {document.iloc[0]['PMID']}, Title: {document.iloc[0]['Title']}, Score: {result[1]}")


In [1]:
# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
model = AutoModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

# Function to generate embeddings
def generate_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()[0]

# Generate embeddings for a column (e.g., Abstract)
df['embedding'] = df['Abstract'].apply(lambda x: generate_embeddings(x))


In [ ]:

es = Elasticsearch("http://localhost:9200")

index_name = "pubmed_articles_embeddings"

# Create index with mapping for dense_vector
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body={
        "mappings": {
            "properties": {
                "title": {"type": "text"},
                "abstract": {"type": "text"},
                "publication_year": {"type": "keyword"},
                "embedding": {"type": "dense_vector", "dims": 768}
            }
        }
    })

# Prepare documents for indexing
docs = [
    {
        "_index": index_name,
        "_id": row["PMID"],
        "_source": {
            "title": row["Title"],
            "abstract": row["Abstract"],
            "publication_year": row["Publication Year"],
            "embedding": row["embedding"].tolist(),
        }
    }
    for index, row in df.iterrows()
]

# Index documents
bulk(es, docs)


In [ ]:
query_vector = generate_embeddings("How does sickness affect intelligence?")

# Script to calculate cosine similarity
script_query = {
    "script_score": {
        "query": {"match_all": {}},
        "script": {
            "source": "cosineSimilarity(params.query_vector, doc['embedding']) + 1.0",
            "params": {"query_vector": query_vector.tolist()}
        }
    }
}

response = es.search(index=index_name, body={"query": script_query})
for hit in response['hits']['hits']:
    print(f"ID: {hit['_id']}, Score: {hit['_score']}, Title: {hit['_source']['title']}")
